In [ ]:
pip install eli5 --user

In [ ]:
import lightgbm as lgb
import numpy as np
import scipy
import random
import os
import shutil
import gc
import sys
import uuid
import functools
import gc
import json
import pandas as pd
import matplotlib
import eli5
import shap
import matplotlib.pyplot as plt
import PIL
from sklearn.metrics import roc_auc_score, roc_curve


fname = ["isEnr", "channel", "tDrift", "avse", "dcr","noise","active mass"]

In [ ]:
bkgdata = np.loadtxt('DS6cal_5.csv', delimiter=',',dtype=np.single)
bkgdata = bkgdata[bkgdata[:,5]<2.32]
np.save("bkgdata.npy",bkgdata)

In [ ]:
bkgdata = np.load("bkgdata.npy")
covar_param = bkgdata[:10000,3:]
plt.figure(figsize=(15,12))
label_name = []
error_matrix = np.corrcoef(np.transpose(covar_param))
plt.imshow(error_matrix,cmap="PiYG",vmin=-1,vmax=1)
ticklabel_array = ["final energy", "avse", "dcr", "tOffset", "nlcblrwfSlope","d2wfnoiseTagNorm"]
plt.yticks(range(len(ticklabel_array)),ticklabel_array)  # Set text labels and properties.
plt.xticks(range(len(ticklabel_array)), ticklabel_array,rotation=90)  # Set text labels and properties.
plt.colorbar()
plt.gcf().subplots_adjust(bottom=0.15)
plt.savefig("covarm.png")

In [ ]:
#Extracting SEP and DEP
bkgdata = np.load("bkgdata.npy")
SEP_energy=2103
DEP_energy=1592
Ewidth =1.0
SEP_test =  bkgdata[(bkgdata[:,3]>(SEP_energy-Ewidth)) & (bkgdata[:,3]<(SEP_energy+Ewidth))]
DEP_test =  bkgdata[(bkgdata[:,3]>DEP_energy-Ewidth) & (bkgdata[:,3]<DEP_energy+Ewidth)]
sig_save = np.delete(DEP_test,3,axis=1)
bkg_save = np.delete(SEP_test,3,axis=1)
# sig_save = np.delete(sig_save,-1,axis=1)
# bkg_save = np.delete(bkg_save,-1,axis=1)
# sig_save = np.delete(sig_save,-1,axis=1)
# bkg_save = np.delete(bkg_save,-1,axis=1)
# sig_save = np.delete(sig_save,-1,axis=1)
# bkg_save = np.delete(bkg_save,-1,axis=1)
print(sig_save.shape,bkg_save.shape)
np.save("sig_raw.npy",sig_save)
np.save("bkg_raw.npy",bkg_save)

In [ ]:
def match_data(signaldata, bkgdata,fname,varname, increment):
    sigindex=[]
    bkgindex=[]
    index = int(fname.index(varname))
    min_select_val = max(np.min(signaldata[:,index]), np.min(bkgdata[:,index]))
    max_select_val = min(np.max(signaldata[:,index]), np.max(bkgdata[:,index]))
    if varname == "dcr":
        min_select_val = max(min_select_val, -10.0)
    elif varname == "noise":
        max_select_val = min(max_select_val, 10.0)
    print(min_select_val, max_select_val)
    for entry_select_val in np.arange(min_select_val, max_select_val+increment, increment):
        sig_in_range = np.where(np.logical_and(signaldata[:,index]>=entry_select_val, signaldata[:,index]<entry_select_val+increment))[0]
        bkg_in_range =np.where(np.logical_and(bkgdata[:,index]>=entry_select_val, bkgdata[:,index]<entry_select_val+increment))[0]
        min_entry = min(len(sig_in_range), len(bkg_in_range))
        if min_entry == 0:
            continue
        sigindex += list(np.random.choice(sig_in_range, min_entry,replace=False))
        bkgindex += list(np.random.choice(bkg_in_range, min_entry,replace=False))
    rg = np.arange(max(min_select_val,-10), max_select_val+increment,increment)
    plt.hist(signaldata[sigindex,index],bins=rg,color="red",linestyle=":",histtype="step",label="DEP")
    plt.hist(bkgdata[bkgindex,index],bins=rg,linestyle="-",label="SEP")
    plt.title("After Matching")
    plt.xlabel("Matched Value[a.u.]")
    plt.legend()
    signaldata = signaldata[sigindex]
    bkgdata = bkgdata[bkgindex]
    print(signaldata.shape,bkgdata.shape)
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()
    return signaldata, bkgdata

In [ ]:
sigsave = np.load("sig_raw.npy")
bkgsave = np.load("bkg_raw.npy")
sigsave, bkgsave = match_data(sigsave,bkgsave,fname, "dcr",0.01)
sigsave, bkgsave = match_data(sigsave,bkgsave,fname,"tDrift",0.1)
sigsave, bkgsave = match_data(sigsave,bkgsave,fname,"channel",1)
sigsave, bkgsave = match_data(sigsave,bkgsave,fname,"noise",0.01)
print(sigsave.shape,bkgsave.shape)
np.save("sig.npy",sigsave)
np.save("bkg.npy",bkgsave)

In [ ]:
signaldata = np.load("sig.npy")
bkgdata = np.load("bkg.npy")


#split signal dataset
test_split = 0.4
indices = np.arange(signaldata.shape[0])
np.random.shuffle(indices)
train_index = indices[int(len(indices)*test_split):]
test_index = indices[:int(len(indices)*test_split)]
signal_train = signaldata[train_index]
signal_test = signaldata[test_index]
siglabel_train = np.ones(signal_train.shape[0])
siglabel_test = np.ones(signal_test.shape[0])

In [ ]:
#split bkg dataset
indices = np.arange(bkgdata.shape[0])
np.random.shuffle(indices)
train_index = indices[int(len(indices)*test_split):]
test_index = indices[:int(len(indices)*test_split)]
bkg_train = bkgdata[train_index]
bkg_test = bkgdata[test_index]
bkglabel_train = np.zeros(bkg_train.shape[0])
bkglabel_test = np.zeros(bkg_test.shape[0])

In [ ]:
#Randomize tran dataset
X_train = np.concatenate([signal_train, bkg_train],axis = 0)
Y_train = np.concatenate([siglabel_train, bkglabel_train],axis = 0)
train_index = np.arange(len(X_train))
np.random.shuffle(train_index)
X_train = X_train[train_index]
Y_train = Y_train[train_index]
X_test = np.concatenate([signal_test, bkg_test],axis = 0)
Y_test = np.concatenate([siglabel_test, bkglabel_test],axis = 0)

In [ ]:
#sample the same amount of events in test dataset
sig_test = np.where(Y_test==1)[0]
bkg_test = np.where(Y_test==0)[0]
min_test_entry = min(len(sig_test), len(bkg_test))
index_test = list(np.random.choice(sig_test, min_test_entry, replace=False)) + list(np.random.choice(bkg_test, min_test_entry, replace=False))
X_test = X_test[index_test]
Y_test = Y_test[index_test]

In [ ]:
#Randomize test dataset
test_index = np.arange(len(X_test))
np.random.shuffle(test_index)
X_test = X_test[test_index]
Y_test = Y_test[test_index]

In [ ]:
#Split test into validation & test dataset
if len(X_test)%2 == 1:
    X_test = X_test[:-1]
    Y_test = Y_test[:-1]
X_val, X_test = np.split(X_test,2)
Y_val, Y_test = np.split(Y_test,2)

valid_x = np.isfinite(X_test[:,4])
X_test = X_test[valid_x]
Y_test = Y_test[valid_x]

In [ ]:
lgb_train = lgb.Dataset(X_train, Y_train,free_raw_data=False, feature_name = fname
)
lgb_eval = lgb.Dataset(X_val, Y_val, reference=lgb_train,free_raw_data=False, feature_name = fname
)
#972
params={"num_iterations": 722, "learning_rate": 0.024617418240526604, "num_leaves": 69, "bagging_freq": 36, "pos_bagging_fraction": 0.5226149147832042, "min_data_in_leaf": 46, "drop_rate": 0.3893833272261452, "min_gain_to_split": 0.2928531948747939, "max_bin": 603, "boosting": "goss", "objective": "binary", "metric": "binary_logloss", "verbose": 0, "neg_bagging_fraction": 1.0}
params["neg_bagging_fraction"] *= params["pos_bagging_fraction"]
evals_result = {}

In [ ]:
gbm = lgb.train(params,lgb_train,valid_sets=lgb_eval,early_stopping_rounds=3,evals_result=evals_result, categorical_feature=["isEnr","channel"])

In [ ]:
shap.initjs()

In [ ]:
#full_output
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
rg=np.arange(0.0,1.0,0.01)
plt.hist(y_pred[Y_test==1], label="Signal", bins=rg, histtype="step", density=True)
plt.hist(y_pred[Y_test==0], label="Background",bins=rg, histtype="step", density=True)
# plt.axvline(x=0.05,linestyle="--",color="black")
# plt.axvline(x=0.5,linestyle="--",color="black")
# plt.axvline(x=0.9,linestyle="--",color="black")
plt.legend()
plt.xlabel("BDT output")
plt.ylabel("% per 0.01 bin(a.u.)")
plt.show()
plt.cla()
plt.clf()
plt.close()

In [ ]:
#Plot one event force plot
# fname = ["isEnr", "channel", "tDrift", "avse", "dcr"]
explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[1], shap_values[1][240,:], fname)
# #print(Y_test[0])

In [ ]:
#Plot all events with BDT output between 0.9 and 1.0 (Dataset A)
Pos_sample = X_test[Y_test == 1]
Neg_sample = X_test[Y_test == 0]
sample = np.concatenate([Pos_sample[:500], Neg_sample[:500]],axis=0)
shap_values = explainer.shap_values(sample)
shap.force_plot(explainer.expected_value[1], shap_values[1][500:,:], fname)

In [ ]:
#Plot all events with BDT output between 0.9 and 1.0 (Dataset A)
#["isEnr", "gain", "tDrift", "avse", "dcr", "channel"]
Pos_sample = X_test[Y_test == 1]
Neg_sample = X_test[Y_test == 0]
sample = np.concatenate([Pos_sample[:500], Neg_sample[:500]],axis=0)
shap_values = explainer.shap_values(sample)
print(shap_values)
shap.summary_plot(shap_values[1], sample,plot_type="dot", feature_names=fname)
shap.dependence_plot(fname.index("avse"), shap_values[1], sample, feature_names=fname)

In [ ]:
def get_roc_param(ytest, ypred):
    ytest = ytest[np.isfinite(ypred)]
    ypred = ypred[np.isfinite(ypred)]
    auc = roc_auc_score(ytest, ypred)
    if auc<0.5:
        ypred = -ypred
    fpr, tpr, thr = roc_curve(ytest, ypred)
    return fpr, tpr, thr, roc_auc_score(ytest, ypred)

def plot_auc(Y_test, y_pred, X_test, elow):
    # Yt_modified = Y_test.flatten()
    # X_avse = X_test[:,4].flatten()
    # X_dcr = X_test[:,5].flatten()
    # X_avse = X_avse[X_dcr<=2.32]
    # Yt_modified = Yt_modified[X_dcr<=2.32]
    # ypred_modified = y_pred[X_dcr<=2.32]

    auc = roc_auc_score(Y_test, y_pred)
    fpr, tpr, thr = roc_curve(Y_test, y_pred)
    fpra, tpra, thra, auca= get_roc_param(Y_test, X_test[:,3])
    
    avsecut = np.argmin(np.abs(thra+1.0))
    bdtcut = np.argmin(np.abs(tpr-tpra[avsecut]))
    
    plt.plot(fpr, tpr, label = 'BDT Classifier (AUC = %.3f, DEP = %.3f, SEP = %.3f)'%(auc, tpr[bdtcut], 1-fpr[bdtcut]))
    plt.plot(fpra, tpra, label = 'A vs. E Classifier (AUC = %.3f, DEP = %.3f, SEP = %.3f)'%(auca, tpra[avsecut], 1-fpra[avsecut]))

    fprt, tprt, thrt, auct = get_roc_param(Y_test, X_test[:,fname.index("tDrift")].flatten())
    plt.plot(fprt, tprt, label = 'tDrift Classifier (AUC = %.3f)'%(auct))

    fprd, tprd, thrd, aucd = get_roc_param(Y_test, X_test[:,fname.index("dcr")].flatten())
    plt.plot(fprd, tprd, label = 'DCR Classifier (AUC = %.3f)'%(aucd))

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    print('False positive rate:',fpr[1], '\nTrue positive rate:',tpr[1])
    print("Elow: ", elow,"BDT cut: ", 1-fpr[np.argmin(np.abs(tpr-0.9))],"A vs. E cut: ", 1-fpra[np.argmin(np.abs(tpra-0.9))])
    print("Elow: ", elow,"BDT cut: ", tpr[np.argmin(np.abs(fpr-0.1))],"A vs. E cut: ", tpra[np.argmin(np.abs(fpra-0.1))])

    plt.show()
    plt.cla()
    plt.clf()
    plt.close()
    return thr[bdtcut], thra[avsecut]

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
bdt_thresh, avse_thresh = plot_auc(Y_test, y_pred, X_test, 0)
sample_selector = (y_pred<bdt_thresh) & (X_test[:,3]>avse_thresh) & (Y_test == 0)
print(bdt_thresh, avse_thresh)
shap_values = explainer.shap_values(X_test[sample_selector])
print(len(X_test[sample_selector]) / len(X_test))
#shap.force_plot(explainer.expected_value[1], shap_values[1][:1000,:], fname)
shap.dependence_plot(fname.index("avse"), shap_values[1], X_test[sample_selector], feature_names=fname)
shap.dependence_plot(fname.index("tDrift"), shap_values[1], X_test[sample_selector], feature_names=fname)

In [ ]:
#BDT output on enriched detector
enriched_selector = X_test[:,0] == 1
Xt = X_test[enriched_selector]
yp = gbm.predict(Xt, num_iteration=gbm.best_iteration)
yt = Y_test[enriched_selector]
rg=np.arange(0.0,1.0,0.01)
plt.hist(yp[yt==1], label="Signal", bins=rg, histtype="step", density=True)
plt.hist(yp[yt==0], label="Background",bins=rg, histtype="step", density=True)
# plt.axvline(x=0.05,linestyle="--",color="black")
# plt.axvline(x=0.5,linestyle="--",color="black")
# plt.axvline(x=0.9,linestyle="--",color="black")
plt.legend()
plt.xlabel("BDT output on Enriched Detector")
plt.ylabel("% per 0.01 bin(a.u.)")
plt.show()
plt.cla()
plt.clf()
plt.close()

In [ ]:
plot_auc(yt, yp, Xt, 0)

In [ ]:
shap_values = explainer.shap_values(X_test[Y_test==1])
shap.plots.heatmap(shap_values, , instance_order=shap_values.sum(1))